In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [2]:
loan = pd.read_csv('Task 3 and 4_Loan_Data.csv')

In [3]:
loan

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0
...,...,...,...,...,...,...,...,...
9995,3972488,0,3033.647103,2553.733144,42691.62787,5,697,0
9996,6184073,1,4146.239304,5458.163525,79969.50521,8,615,0
9997,6694516,2,3088.223727,4813.090925,38192.67591,5,596,0
9998,3942961,0,3288.901666,1043.099660,50929.37206,2,647,0


In [4]:
recoverRate = 0.1

In [5]:
#1. spliting the data into train and test data: using 60/40 -- 60% training & 40% 
X = loan.drop(['default'], axis=1)
y = loan['default'] # Target variable: default (1 if defaulted, 0 otherwise)

#spliting data into training and testing sets
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size= 0.4, random_state=34) 

scaler = StandardScaler()
xtrain_scaled = scaler.fit_transform(Xtrain)
xtest_scaled = scaler.transform(Xtest)

In [6]:
xtrain_scaled #scaled training data

array([[ 0.99575889, -0.25665542, -0.45989617, ..., -0.09892904,
         0.29460452,  0.42742093],
       [ 1.34208885, -0.82976019,  0.45617466, ...,  0.68214917,
        -0.3439891 ,  1.10305064],
       [-0.4741853 , -0.25665542,  0.81942656, ...,  0.73755359,
         0.29460452,  0.92178413],
       ...,
       [-1.42944407, -0.82976019, -1.36912535, ..., -1.75157641,
        -0.3439891 , -0.74257198],
       [-1.5324172 , -0.25665542,  1.15293777, ...,  1.40989687,
         0.93319814,  0.32854829],
       [-1.3045757 , -0.82976019,  0.06813382, ...,  0.39881893,
         0.93319814,  1.69628648]])

In [7]:
# try random forest classifier to predict default
rfClassifier = RandomForestClassifier(random_state=34)
rfClassifier.fit(xtrain_scaled, ytrain)

# predicting default probabilities on the test set
yPredictP_rf = rfClassifier.predict_proba(xtest_scaled)[:,1]

# Evaluating the model's performance using ROC-AUC score
roc_auc_rf = roc_auc_score(ytest, yPredictP_rf)

In [8]:
roc_auc_rf

0.9997592212127956

ROC_AUC score is exceptional for the Random Forest Classifier, where the ROC_AUC ranges from 0 to 1. 

In [9]:
# try a different method: Decision Tree
from sklearn.tree import DecisionTreeClassifier

In [10]:
#training the decision tree classifier
dtClassifier = DecisionTreeClassifier(random_state=34)
dtClassifier.fit(xtrain_scaled, ytrain)

# predicting default prob on test set
yPredictP_dt = dtClassifier.predict_proba(xtest_scaled)[:,1]

#evaluating the model's performance using ROC_AUC socre
roc_auc_dt = roc_auc_score(ytest, yPredictP_dt)

In [11]:
roc_auc_dt

0.9900602418279679

In [12]:
# try gradient boosting
from sklearn.ensemble import GradientBoostingClassifier

In [13]:
# training
gbClassifier = GradientBoostingClassifier(random_state=34)
gbClassifier.fit(xtrain_scaled, ytrain)

# predict
yPredictP_gb = gbClassifier.predict_proba(xtest_scaled)[:,1]

#evaluation 

roc_auc_gb = roc_auc_score(ytest, yPredictP_gb)


In [14]:
roc_auc_gb

0.9997618851483137

In [15]:
import xgboost as xgb

In [16]:
xgbClassifier = xgb.XGBClassifier(
    objective = 'binary:logistic', 
    eval_metric = 'auc', 
    use_label_encoder = False, # To avoid a deprecation warning
    random_state = 34
 )

xgbClassifier.fit(xtrain_scaled, ytrain)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=34, ...)

In [17]:
roc_auc_xgb = roc_auc_score(ytest, xgbClassifier.predict_proba(xtest_scaled)[:, 1])

roc_auc_xgb

0.9999213114431604

In [18]:
# cross-validation to detect the overfitting and a more robust estimate of a model's performance
from sklearn.model_selection import cross_val_score

In [19]:
# cross validation for Random Forest
cv_rf = cross_val_score(rfClassifier, xtrain_scaled, ytrain, cv=5, scoring='roc_auc')
print(f"Random Forest - CV Mean: {cv_rf.mean()}, CV Std: {cv_rf.std()}")

Random Forest - CV Mean: 0.9996887755102042, CV Std: 0.00022858185679652617


In [20]:
# cross validation for Decision Tree
cv_dt = cross_val_score(dtClassifier, xtrain_scaled, ytrain, cv=5, scoring='roc_auc')
print(f"Decision Tree - DT Mean: {cv_dt.mean()}, DT Std: {cv_dt.std()}")

Decision Tree - DT Mean: 0.9879591836734694, DT Std: 0.005336217958534341


In [21]:
# cross validation for Gradient Boosting
cv_gb = cross_val_score(gbClassifier, xtrain_scaled, ytrain, cv=5, scoring='roc_auc')
print(f"Gradient Boosting - GB Mean: {cv_gb.mean()}, GB Std: {cv_gb.std()}")

Gradient Boosting - GB Mean: 0.9996971243042673, GB Std: 0.0002576679857541509


In [22]:
# cross validation for XGBoost
cv_xgb = cross_val_score(xgbClassifier, xtrain_scaled, ytrain, cv=5, scoring='roc_auc')
print(f"XGBoosting - XGB Mean: {cv_xgb.mean()}, XGB Std: {cv_xgb.std()}")

XGBoosting - XGB Mean: 0.9998293135435992, XGB Std: 0.0001677830418796913


The cross validation shows that both random forest and gradient boosting hold pretty good result, whose results are very close to 1.
Although the random forest would give a faster prediction speed, and gradient boosting for the better performance than random forest, the XGBoost would provide a better performance and a faster speed because of its optimiztion and parallel processing.  

The expected loss is calculated as: 

$$ Expected\ loss = \mathbb{P}(Default) * loan\ amount * (1 - Recovery\ Rate) $$

In [23]:
def predictDefaultP(loanFestures, model, scaler):
    featureScaled = scaler.transform(loanFestures)
    
    defaultP = model.predict_proba(featureScaled)[:,1]
    
    return defaultP
    

In [24]:
prob_default_test = predictDefaultP(Xtest, xgbClassifier, scaler)

Xtest['Prob of default'] = prob_default_test

In [25]:
prob_default_train = predictDefaultP(Xtrain, xgbClassifier,scaler)
Xtrain['Prob of default'] = prob_default_train

In [26]:
def Eloss(DefaultP, loanAmt, recoveryRate = 0.1):
    
    return DefaultP * loanAmt * (1-recoveryRate)

In [29]:
expected_loss_test = Eloss(prob_default_test, Xtest['loan_amt_outstanding'].values[0])
Xtest['expected_loss'] = expected_loss_test

In [30]:
expected_loss_train = Eloss(prob_default_train, Xtrain['loan_amt_outstanding'].values[0])
Xtrain['expected_loss'] = expected_loss_train

Task 4

In [31]:
# quantization of FICO score: split FICO scores into bins
def quantizeFICO(fico):
    if 300 <= fico <= 579:
        return 'Poor'
    elif 580 <= fico <= 669:
        return 'Fair'
    elif 670 <= fico <= 739:
        return 'Good'
    elif 740 <= fico <= 799:
        return 'Very good'
    elif 800 <= fico <= 850:
        return 'Excellent'
    else:
        return 'invalid'

In [ ]:
# apply quantization to the data:
loan['fico_category'] = loan['fico_score'].apply(quantizeFICO)

In [32]:
loan

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0
...,...,...,...,...,...,...,...,...
9995,3972488,0,3033.647103,2553.733144,42691.62787,5,697,0
9996,6184073,1,4146.239304,5458.163525,79969.50521,8,615,0
9997,6694516,2,3088.223727,4813.090925,38192.67591,5,596,0
9998,3942961,0,3288.901666,1043.099660,50929.37206,2,647,0


In [38]:
Xtest['fico_category'] = Xtest['fico_score'].apply(quantizeFICO)
Xtest

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,Prob of default,expected_loss,fico_category
2390,4254020,1,3033.502993,5459.760802,58145.40500,5,598,0.000012,0.033632,Fair
6803,4153925,1,5131.838746,10273.686330,86735.09997,4,575,0.000004,0.011986,Poor
3226,7240749,0,1599.932207,1077.329553,26191.38070,2,697,0.000355,0.968765,Good
8630,1771847,0,3939.469863,3022.037280,59835.22980,3,625,0.000015,0.041654,Fair
4898,6631761,0,3857.340486,3290.320625,77903.38542,3,634,0.000006,0.017426,Fair
...,...,...,...,...,...,...,...,...,...,...
3506,2397436,3,3669.635064,14239.425930,74670.59076,4,655,0.000358,0.978386,Fair
99,1638898,1,3707.461716,4634.160556,57597.45492,5,633,0.000010,0.028008,Fair
4431,3679461,0,3286.939510,7242.783127,64700.55843,3,613,0.000008,0.022651,Fair
2212,8660570,2,5606.289442,13883.832690,91723.19680,7,657,0.000006,0.015714,Fair


In [39]:
Xtrain['fico_category'] = Xtrain['fico_score'].apply(quantizeFICO)
Xtrain

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,Prob of default,expected_loss,fico_category
7803,7291395,1,3514.373898,5534.786569,68134.16335,5,663,0.000004,0.012883,Fair
2912,8086988,0,4808.997289,846.620712,83878.82050,4,704,0.000002,0.005588,Good
8244,3914623,1,5322.357599,10089.678410,84995.64043,5,693,0.000002,0.007766,Good
3695,3469901,1,3808.427293,6372.428393,63076.46622,7,656,0.000002,0.005415,Fair
2629,1337395,5,4271.314690,22756.281030,83475.30929,4,438,0.999985,3162.888428,Poor
...,...,...,...,...,...,...,...,...,...,...
5667,4437459,0,1712.912777,1209.698471,29131.62730,4,605,0.000062,0.196178,Fair
324,8099678,2,3973.657805,9369.089493,63687.96336,5,725,0.000007,0.023369,Good
3157,1720192,0,2229.419378,3043.967766,34820.76913,4,592,0.000079,0.249983,Fair
5993,1483641,1,5793.687316,9207.734040,98548.46301,6,657,0.000002,0.006388,Fair


In [36]:
from scipy.optimize import minimize

In [37]:
#log likelihood function
def neg_LogLikelihood(rep, bucket):
    mu = rep
    sigma = bucket['Prob of default'].std()
    logLikelihood = -0.5 * np.sum(((bucket['Prob of default'] - mu) ** 2 )/(sigma ** 2)) - len(bucket) * np.log(sigma * np.sqrt(2 * np.pi))
    return -logLikelihood

In [41]:
# Labels of FICO socres 
labels = ['Poor', 'Fair', 'Good', 'Very good', 'Excellent']
repValues = {}
for dataset in [Xtest, Xtrain]:
    for label in labels: 
        bucket = dataset[dataset['fico_category'] == label]
        
        if bucket.empty:
            continue
        
        result = minimize(neg_LogLikelihood, x0= np.mean(bucket['Prob of default']), args=(bucket))
        repValues[label] = result.x[0]

In [42]:
#dynamic programming for rating

ratings = ['A', 'B', 'C', 'D', 'E']

sortedlabels = sorted(repValues.keys(), key=lambda x: repValues[x])

ratingMap = {label: rating for label, rating in zip(sortedlabels, ratings)}

print(ratingMap)

{'Very good': 'A', 'Excellent': 'B', 'Good': 'C', 'Fair': 'D', 'Poor': 'E'}


In [45]:
for label, value in repValues.items():
    print(f"{label}: {value:.4f}")

Poor: 0.4296
Fair: 0.1698
Good: 0.0747
Very good: 0.0261
Excellent: 0.0500
